# Pure S3 Path Manipulation

## What is Pure S3 Path

A Pure S3 Path is a Python object that represents an AWS S3 bucket, object, or folder. However, it's important to note that a Pure S3 Path object does not make any calls to the AWS API, nor does it imply the existence of the corresponding S3 object. Rather, it's a lightweight abstraction that allows you to work with S3 paths in a Pythonic, object-oriented manner without incurring any network overhead.

In [3]:
from s3pathlib import S3Path

s3path = S3Path("s3://bucket/folder/file.txt")
print(s3path)

S3Path('s3://bucket/folder/file.txt')


## Construct an S3 Path object in Python

### From bucket, and key parts

In a file system, you typically use a file path like ``C:\\Users\username\file.txt`` on Windows or ``/Users/username/file.txt`` on a POSIX system. It's similarly intuitive to construct an S3 Path from a string.

In [4]:
# construct from bucket, key parts
s3path = S3Path("bucket", "folder", "file.txt")
s3path

S3Path('s3://bucket/folder/file.txt')

In [5]:
# construct from full path also works
s3path = S3Path("bucket/folder/file.txt")
s3path

S3Path('s3://bucket/folder/file.txt')

S3 uses ``/`` as a [delimiter to organize and browse your keys hierarchically](https://docs.aws.amazon.com/AmazonS3/latest/userguide/using-prefixes.html). With ``s3pathlib``, the delimiter is handled intelligently.

In [6]:
s3path = S3Path("bucket", "/folder/", "/file.txt")
s3path

S3Path('s3://bucket/folder/file.txt')

### From S3 URI

[S3 URI](https://repost.aws/questions/QUFXlwQxxJQQyg9PMn2b6nTg/questions/QUFXlwQxxJQQyg9PMn2b6nTg/what-is-s3-uri-in-simple-storage-service?) is the unique resource identifier within the context of the S3 protocol. They follow this naming convention: ``s3://bucket-name/key-name``. You can create an S3 Path from S3 URI.

In [10]:
s3path = S3Path("s3://bucket/folder/file.txt")
s3path

S3Path('s3://bucket/folder/file.txt')

In [11]:
s3path = S3Path.from_s3_uri("s3://bucket/folder/file.txt")
s3path

S3Path('s3://bucket/folder/file.txt')

### From S3 ARN

[S3 ARN](https://docs.aws.amazon.com/AmazonS3/latest/userguide/s3-arn-format.html) is the Amazon Resource Name of an S3 resources. They follow this naming convention: ``arn:aws:s3:::bucket_name/key_name``. You can create an S3 Path from S3 ARN.

In [12]:
s3path = S3Path("arn:aws:s3:::bucket/folder/file.txt")
s3path

S3Path('s3://bucket/folder/file.txt')

In [13]:
s3path = S3Path.from_s3_arn("arn:aws:s3:::bucket/folder/file.txt")
s3path

S3Path('s3://bucket/folder/file.txt')

## S3 Path Types

S3 Path is a logical concept that can represent different types of AWS S3 concepts. Here is the list of S3 Path types:

1. **Classic S3 object**: represents an S3 object, such as ``s3://bucket/folder/file.txt``.
2. **Logical S3 directory**: represents an S3 directory, such as ``s3://bucket/folder/``.
3. **S3 bucket**: represents an S3 bucket, such as ``s3://bucket/``
4. **Void Path**: denotes the absence of any bucket or key, essentially representing a blank slate, no bucket, no key, no nothing.
5. **Relative Path**: represents a path relative to another S3 Path. For example, the relative path from ``s3://bucket/folder/file.txt`` to ``s3://bucket/`` is simply ``folder/file.txt``. A relative path can be joined with another S3 Path to create a new S3 Path. Importantly, any concrete path joined with a void path will result in the original concrete path.
6. **Concrete Path**: represents an S3 Path that refers to a concrete object in the S3 storage system. This includes classic S3 object paths, logical S3 directory paths, and S3 bucket paths. Any concrete path joined with a relative path will result in another concrete path.

### Classic S3 object

Similar to a file on your local laptop, an [S3 object](https://docs.aws.amazon.com/AmazonS3/latest/userguide/UsingObjects.html) stores your data. At any given moment, it could be just a pointer, and the object doesn't have to exist in S3.

In [14]:
s3path = S3Path("s3://bucket/folder/file.txt")
s3path

S3Path('s3://bucket/folder/file.txt')

### Logical S3 Directory

Since [AWS S3 is an object storage system](https://aws.amazon.com/s3/), not a file system, directories are only a logical concept in AWS S3. AWS uses / as the path delimiter in S3 keys. There are two types of directories in AWS S3:

- Hard directory: When you create a folder in the S3 console, it creates a special object without any content (an empty string) with the ``/`` character at the end of the key. You can see the folder as an object in the [list_objects](https://boto3.amazonaws.com/v1/documentation/api/latest/reference/services/s3/client/list_objects_v2.html) API response.
- Soft directory: This type of directory does not actually exist; it is [a virtual concept used to help organize your objects in a folder](https://docs.aws.amazon.com/AmazonS3/latest/userguide/using-folders.html). For example, if you have an S3 object like ``s3://bucket/folder/file.txt``, then the ``s3://bucket/folder/`` path is a soft folder. Although you can see it in the S3 console, it does not actually exist.

You can create a S3 directory from string, URI, ARN

In [15]:
s3path = S3Path("bucket", "folder/")
s3path

S3Path('s3://bucket/folder/')

In [16]:
s3path = S3Path("s3://bucket/folder/")
s3path

S3Path('s3://bucket/folder/')

In [17]:
s3path = S3Path("arn:aws:s3:::bucket/folder/")
s3path

S3Path('s3://bucket/folder/')

In [18]:
s3path = S3Path("bucket", "folder").to_dir()
s3path

S3Path('s3://bucket/folder/')

###


.. code-block:: python

    # construct from string, auto join parts
    >>> S3Path("bucket", "folder", "subfolder/")
    S3Path('s3://bucket/folder/subfolder/')

``/`` **is a Python Operator Override Syntax Sugar, it is the Pythonic way to Construct a Path**.

.. code-block:: python

    >>> s3path_bucket = S3Path("bucket")
    >>> s3path_bucket / "folder/"
    S3Path('s3://bucket/folder/')

    # You can chain "/" together
    >>> s3path_bucket / "folder" / "file.txt"
    S3Path('s3://bucket/folder/file.txt')

    # it work with list too
    >>> s3path_bucket / ["folder", "file.txt"]
    S3Path('s3://bucket/folder/file.txt')

It works with :ref:`relative-path` too.

**Summary**:

- The **first** non-void argument defines the ``bucket``
- The **last** non-void argument defines whether it is a ``directory`` or a ``object``

**From S3 URI or S3 ARN**

S3 URI is a unique resource identifier that uniquely locate a S3 bucket, S3 object or S3 directory. ARN is Amazon Resource Namespace that also locate a unique AWS Resource such as a S3 bucket, S3 object or S3 directory.

.. code-block:: python

    >>> S3Path.from_s3_uri("s3://bucket/folder/file.txt")
    S3Path('s3://bucket/folder/file.txt')

    >>> S3Path.from_s3_arn("arn:aws:s3:::bucket/folder/file.txt")
    S3Path('s3://bucket/folder/file.txt')

## Construct S3 Path object

In [2]:
# construct from string, auto join parts
s3path = S3Path("bucket", "folder", "file.txt")
s3path

S3Path('s3://bucket/folder/file.txt')

In [3]:
s3path.bucket

'bucket'

In [4]:
s3path.key

'folder/file.txt'

In [5]:
s3path.uri

's3://bucket/folder/file.txt'

## Another one


<div class="alert alert-warning">

Hello

asdfa
</div>

<div class="admonition important">

<p class="admonition-title">
Hello
</p>
asdfa
</div>


In [6]:
print(123)

123
